In [4]:
import pandas as pd
import numpy as np
import glob, os

# Get CSV files list from a folder
path = 'data/crashes_data'
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file, encoding = 'utf-16') for file in csv_files)


#encoding moest hier gedefinieerd worden anders kreeg ik een error, https://stackoverflow.com/questions/50342517/unicodedecodeerror-reading-a-csv-file

# Concatenate all DataFrames
df   = pd.concat(df_list, ignore_index=True)

In [2]:
df = df[["Date","Package Name","Daily Crashes","Daily ANRs"]]

In [5]:
df.isna().sum()/len(df)*100

#laat het percentage NaN values zijn.

df = df.groupby('Date')[['Daily Crashes']].sum()

print(df)

            Daily Crashes
Date                     
2021-06-01             15
2021-06-02             12
2021-06-03             20
2021-06-04             13
2021-06-05             14
...                   ...
2021-12-27             64
2021-12-28             60
2021-12-29             37
2021-12-30             46
2021-12-31             50

[214 rows x 1 columns]


In [9]:
# Bokeh Libraries
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models.tools import HoverTool
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral5
from bokeh.transform import factor_cmap

# The figure will be rendered inline in my Jupyter Notebook
output_notebook()

# Example figure
source = ColumnDataSource(df)
dates = source.data['Date'].tolist()
p = figure(x_range=dates)


p.vbar(x='Date', top='Daily Crashes', source=source, width=0.70)

p.title.text ='Daily Crashes'
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Crashes'

# See what it looks like
show(p)

# https://programminghistorian.org/en/lessons/visualizing-with-bokeh

Loading BokehJS ...